# Percentages of flights belonging to a distance group that halved a delay

In this notebook we compute the percentage of flights belonging to a given "distance group" that were able to halve their departure delays by the time they arrived at their destinations. Distance groups assort flights by their total distance in miles. Flights with distances that are less than 200 miles belong in group 1, flights with distances that are between 200 and 399 miles belong in group 2, flights with distances that are between 400 and 599 miles belong in group 3, and so on. The last group contains flights whose distances are between 2400 and 2599 miles.


So basically:

- By the _Distance_ attribute we can infer the flight group:

\begin{equation}
FlightGroup = \left( Distance // 200 \right) + 1
\end{equation}

- For each flight we can say if it has halved or not its delay:


\begin{equation}
    DelayHalved=
    \begin{cases}
      yes, & \text{if}\ \left(DepDealy \neq 0\right) \quad \textrm{and} \quad \left(ArrDelay <= \frac{DepDealy}{2} \right)\\
      no, & \text{otherwise}
    \end{cases}
\end{equation}


- So, for each flight group we can create two sets of flight: those that have halved their delays and the others

### Assumptions

- The last group, as the assignment states, is group ( 2599 // 200 ) + 1 = 13
- Each flight not belonging to any group is not considered

#### Execution mode

If development mode is turned on, all the calculations will be performed on a reduced instance of the data set.

This allows to *save computation time* while developing.

In [1]:
### development mode
# False when performing real analytics
DEV = True

### threads to be used to run spark worker nodes locally
spark_local_threads = 6  

### max number of flights groups
max_flight_group = 13

#### Find and import Apache Spark on this machine

In [2]:
import findspark
findspark.init()

In [3]:
# imports
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
import warnings; warnings.simplefilter('ignore')

#### Create a new session

In [4]:
# Build a Spark SQL Session for DataFrames
master = 'local[{}]'.format(spark_local_threads)
appName = 'Airport Distance Groups Percentages'
spark = SparkSession \
    .builder \
    .appName(appName) \
    .master(master) \
    .getOrCreate()

#### Load the data

Try to load the optimized *parquet* format data set. If *parquet* data set is not found, load full compressed data sets, reduce and save them.

In [5]:
from preprocessing_utils import *

if DEV:
    # DEV preprocessing
    perform_DEV_dataset_preprocessing(spark)
else:
    # Production preprocessing
    perform_dataset_preprocessing(spark)

--------- DEV mode ON ---------
Starting preprocessing of ../dataset/1994.csv.bz2
Preprocessing NOT performed.
Preprocessed dataset already exists: ../dataset/preprocessed_dataset_1994.parquet



In [6]:
# Load the parquet dataset

if DEV:
    # Load DEV dataset
    df = load_DEV_preprocessed_dataset(spark)
else:
    # Load production dataset
    df = load_preprocessed_dataset(spark)

--------- DEV mode ON ---------
Peprocessed dataset loaded.
../dataset/preprocessed_dataset_1994.parquet


In [7]:
# Explore the data
df.printSchema()
print(df.head(2))
df.dtypes

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- 

[('Year', 'int'),
 ('Month', 'int'),
 ('DayofMonth', 'int'),
 ('DayOfWeek', 'int'),
 ('DepTime', 'string'),
 ('CRSDepTime', 'int'),
 ('ArrTime', 'string'),
 ('CRSArrTime', 'int'),
 ('UniqueCarrier', 'string'),
 ('FlightNum', 'int'),
 ('TailNum', 'string'),
 ('ActualElapsedTime', 'string'),
 ('CRSElapsedTime', 'int'),
 ('AirTime', 'string'),
 ('ArrDelay', 'int'),
 ('DepDelay', 'int'),
 ('Origin', 'string'),
 ('Dest', 'string'),
 ('Distance', 'int'),
 ('TaxiIn', 'string'),
 ('TaxiOut', 'string'),
 ('Cancelled', 'int'),
 ('CancellationCode', 'string'),
 ('Diverted', 'int'),
 ('CarrierDelay', 'string'),
 ('WeatherDelay', 'string'),
 ('NASDelay', 'string'),
 ('SecurityDelay', 'string'),
 ('LateAircraftDelay', 'string')]

In [8]:
# Select useful attributes
df = df.select(df['ArrDelay'], df['DepDelay'], df['Distance'])

N_before = df.count() # 5180048
# Drop null. Or maybe no! Maybe we can have the percentage of flights with missing values.
df = df.dropna()
N_after = df.count() # 5180048

print("# of rows \n\t BEFORE: {}\n\t  AFTER: {}\n\tDROPPED: {}".format(N_before, N_after, N_before-N_after))

df.show(5)

# of rows 
	 BEFORE: 5180048
	  AFTER: 5078411
	DROPPED: 101637
+--------+--------+--------+
|ArrDelay|DepDelay|Distance|
+--------+--------+--------+
|       0|       0|     487|
|      -2|       5|     487|
|      -4|       0|     487|
|      -3|       0|     487|
|      -5|       0|     487|
+--------+--------+--------+
only showing top 5 rows



## FlightGroup

#### Which is the maximum Distance?

In [9]:
if DEV and 0:
    max_row = df.rdd.max(key=lambda x: int(x["Distance"]))

    print(max_row) # Row(ArrDelay='-13', DepDelay='6', Distance='4502')

In [10]:
# df.dtypes

In [11]:
# Calculate group
df = df.withColumn("Group", F.floor(df.Distance / 200) + 1)

#df.show(100)
#df.show()

#### Cut out all the groups above the considered limit

All flights that were categorized in groups above 13, will be dropped.

In [12]:
N_before = df.count()
# cut out all the groups above the considered limit
df = df.filter(df.Group <= max_flight_group)
N_after = df.count()

print("# of rows \n\t BEFORE: {}\n\t  AFTER: {}\n\tDROPPED: {}".format(N_before, N_after, N_before-N_after))

# of rows 
	 BEFORE: 5078411
	  AFTER: 5062226
	DROPPED: 16185


## Delay was halved or not?

In [13]:
def delay_halved(args):
    dep_delay, arr_delay = args[0], args[1]
    
    # case in which the flight was departed on time
    # it cannot happen that it is able to halve its delay. There's no delay!
    if dep_delay == 0:
        return 'no'
    
    if arr_delay <=  (dep_delay / 2):
        return 'yes'
    
    # delay was not halved: arr_delay > dep_delay / 2
    return 'no'

delay_halved = F.udf(delay_halved, StringType())

# Calculate if the delay was halved or not
df = df.withColumn("DelayHalved", delay_halved(F.array('DepDelay', 'ArrDelay')))

df.show()

+--------+--------+--------+-----+-----------+
|ArrDelay|DepDelay|Distance|Group|DelayHalved|
+--------+--------+--------+-----+-----------+
|       0|       0|     487|    3|         no|
|      -2|       5|     487|    3|        yes|
|      -4|       0|     487|    3|         no|
|      -3|       0|     487|    3|         no|
|      -5|       0|     487|    3|         no|
|      15|      24|     487|    3|         no|
|      -1|       0|     487|    3|         no|
|     -10|       0|     487|    3|         no|
|       5|       0|     487|    3|         no|
|       2|       3|     487|    3|         no|
|      13|      15|     487|    3|         no|
|       3|       7|     487|    3|        yes|
|       1|       5|     487|    3|        yes|
|      -5|       0|     487|    3|         no|
|      -2|       0|     487|    3|         no|
|      -3|       0|     487|    3|         no|
|     -10|       0|     487|    3|         no|
|       4|      15|     487|    3|        yes|
|       0|   

In [14]:
# create count support column
df = df.withColumn("DelayHalvedCount", F.lit(1))

df_perc = df.groupBy('Group', 'DelayHalved')                                  \
            .sum('DelayHalvedCount')                                          \
            .withColumnRenamed('sum(DelayHalvedCount)', 'DelayHalvedCount')   \
            .orderBy('Group', 'DelayHalved')

# data frame rows should be only 13*2 rows, feasible to retrieve a pandas.DataFrame
pdf_perc = df_perc.toPandas()

df_perc.show(30)

+-----+-----------+----------------+
|Group|DelayHalved|DelayHalvedCount|
+-----+-----------+----------------+
|    1|         no|          338014|
|    1|        yes|          199997|
|    2|         no|          970216|
|    2|        yes|          470533|
|    3|         no|          576346|
|    3|        yes|          345553|
|    4|         no|          389029|
|    4|        yes|          241570|
|    5|         no|          322100|
|    5|        yes|          212378|
|    6|         no|          208610|
|    6|        yes|          132113|
|    7|         no|           91235|
|    7|        yes|           57649|
|    8|         no|          108068|
|    8|        yes|           69699|
|    9|         no|           72380|
|    9|        yes|           52282|
|   10|         no|           35101|
|   10|        yes|           30609|
|   11|         no|           24798|
|   11|        yes|           17400|
|   12|         no|           18589|
|   12|        yes|           13968|
|

In [15]:
# calculate total number of flights just as a cross check 
df_total = df.groupBy('Group', 'DelayHalvedCount')                              \
                .sum('DelayHalvedCount')                                        \
                .drop('DelayHalvedCount')                                       \
                .withColumnRenamed('sum(DelayHalvedCount)', 'DelayHalvedCount') \
                .orderBy('Group')

# data frame rows should be only 13 rows, feasible to retrieve a pandas.DataFrame
pdf_total = df_total.toPandas()

df_total.show(15)

+-----+----------------+
|Group|DelayHalvedCount|
+-----+----------------+
|    1|          538011|
|    2|         1440749|
|    3|          921899|
|    4|          630599|
|    5|          534478|
|    6|          340723|
|    7|          148884|
|    8|          177767|
|    9|          124662|
|   10|           65710|
|   11|           42198|
|   12|           32557|
|   13|           63989|
+-----+----------------+



In [16]:
# Check sum of yes and no for halved delay correspond to the total number of flights of the group
for i, row in pdf_total.iterrows():
    row_p_no = pdf_perc.iloc[i*2]
    row_p_yes = pdf_perc.iloc[i*2+1]
    print("asserting: {} == {} + {}".format(row['DelayHalvedCount'], row_p_no['DelayHalvedCount'], row_p_yes['DelayHalvedCount']))
    assert(row['DelayHalvedCount'] == row_p_no['DelayHalvedCount'] + row_p_yes['DelayHalvedCount'])

asserting: 538011 == 338014 + 199997
asserting: 1440749 == 970216 + 470533
asserting: 921899 == 576346 + 345553
asserting: 630599 == 389029 + 241570
asserting: 534478 == 322100 + 212378
asserting: 340723 == 208610 + 132113
asserting: 148884 == 91235 + 57649
asserting: 177767 == 108068 + 69699
asserting: 124662 == 72380 + 52282
asserting: 65710 == 35101 + 30609
asserting: 42198 == 24798 + 17400
asserting: 32557 == 18589 + 13968
asserting: 63989 == 37059 + 26930
